<a href="https://colab.research.google.com/github/collinjennings/detectiveLLMs/blob/main/chunkingTextSummarization.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Workflow for Summarizing Chunked Texts


In [1]:
import glob
from collections import defaultdict
import re
import os
import numpy as np
import pandas as pd

In [2]:
from google.colab import drive
drive.mount('/content/drive')
%cd '/content/drive/MyDrive/Colab Notebooks'

Mounted at /content/drive
/content/drive/MyDrive/Colab Notebooks


In [ ]:
!pip install transformers

In [3]:
from transformers import AutoTokenizer, AutoModelForCausalLM

import math
import os

os.environ['CUDA_VISIBLE_DEVICES'] = '0,1,2,3'

In [4]:
from huggingface_hub import login
login(token="")

The token has not been saved to the git credentials helper. Pass `add_to_git_credential=True` in this function directly or `--add-to-git-credential` if using via `huggingface-cli` if you want to set the git credential as well.
Token is valid (permission: fineGrained).
Your token has been saved to /root/.cache/huggingface/token
Login successful


In [11]:
def _summarize(text,max_tokens, model, tokenizer):
    B_INST, E_INST = "[INST] ", " [/INST]"
    prompt = f"{B_INST}Write a concise summary of the following text:\n\n[TEXT_START]\n\n{text}\n\n[TEXT_END]\n\n{E_INST}"
    inputs = tokenizer( prompt,return_tensors="pt")
    outputs = model.generate(**inputs, max_new_tokens=max_tokens, use_cache=True, do_sample=True,temperature=0.2, top_p=0.95)
    prompt_length = inputs['input_ids'].shape[1]
    summary = tokenizer.decode(outputs[0][prompt_length:-1])
    return summary

def chunk(tokens, max_token_length,tokenizer):
    token_length = len(tokens)
    k = math.ceil(token_length /max_token_length)
    chunk_sizes = [token_length // k + (1 if x < token_length % k else 0)  for x in range (k)]
    #print(token_length, k, chunk_sizes)
    last = 1
    texts =[]
    for l in chunk_sizes:
        sub_sequence_ids = tokens[last: last+l]
        last +=l
        texts.append(tokenizer.decode(sub_sequence_ids))
    return texts

In [6]:
tokenizer = AutoTokenizer.from_pretrained("mistralai/Mistral-7B-Instruct-v0.1", model_max_length = 1000, cache_dir= "/data/summaries/")
#tokens = tokenizer.encode( content)#,return_tensors="pt")
#len(tokens)

/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_token.py:89: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


tokenizer_config.json:   0%|          | 0.00/2.10k [00:00<?, ?B/s]

tokenizer.model:   0%|          | 0.00/493k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/1.80M [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/414 [00:00<?, ?B/s]

In [13]:
#cache_dir= "/data/summaries/"
#model = AutoModelForCausalLM.from_pretrained("mistralai/Mistral-7B-Instruct-v0.1", pad_token_id = tokenizer.eos_token_id,cache_dir= cache_dir, device_map="auto")
#model.half()
tokens = tokenizer.encode( content)#,return_tensors="pt")
summary = ""
max_token_length = 500
complete_runthroughs = 0
#while len(tokens) > max_token_length:
texts = chunk(tokens, max_token_length, tokenizer)
summaries = []
for text in texts:
  sub_summary = _summarize(text, max_token_length, model, tokenizer)
  summaries.append(sub_summary)
        #summary = " ".join(summaries)

Setting `pad_token_id` to `eos_token_id`:2 for open-end generation.
/usr/local/lib/python3.10/dist-packages/transformers/generation/utils.py:1885: UserWarning: You are calling .generate() with the `input_ids` being on a device type different than your model's device. `input_ids` is on cpu, whereas the model is on cuda. You may experience unexpected behaviors or slower generation. Please make sure that you have put `input_ids` to the correct device by calling for example input_ids = input_ids.to('cuda') before running `.generate()`.
  warnings.warn(
Setting `pad_token_id` to `eos_token_id`:2 for open-end generation.
Setting `pad_token_id` to `eos_token_id`:2 for open-end generation.
Setting `pad_token_id` to `eos_token_id`:2 for open-end generation.
Setting `pad_token_id` to `eos_token_id`:2 for open-end generation.
Setting `pad_token_id` to `eos_token_id`:2 for open-end generation.
Setting `pad_token_id` to `eos_token_id`:2 for open-end generation.
Setting `pad_token_id` to `eos_token_

In [1]:
len(summaries)

NameError: name 'summaries' is not defined

In [14]:
outf= open("sample_out.txt","w")
outf.write(summaries)

TypeError: write() argument must be str, not list

In [12]:
def summarize(text, cache_dir = None):
    max_token_length = 1000

    tokenizer = AutoTokenizer.from_pretrained("mistralai/Mistral-7B-Instruct-v0.1", model_max_length = max_token_length, cache_dir= cache_dir)
    model = AutoModelForCausalLM.from_pretrained("mistralai/Mistral-7B-Instruct-v0.1", pad_token_id = tokenizer.eos_token_id,cache_dir= cache_dir, device_map="auto")
    model.half()

    tokens = tokenizer.encode( text)#,return_tensors="pt")
    summary = ""
    complete_runthroughs = 0
    while len(tokens) > max_token_length:
        texts = chunk(tokens, max_token_length, tokenizer)

        summaries = []
        for text in texts:
            sub_summary = _summarize(text, max_token_length, model, tokenizer)
            summaries.append(sub_summary)
        summary = " ".join(summaries)

        tokens = tokenizer.encode(summary)
        complete_runthroughs+=1
        print(f"run through entire text doc {complete_runthroughs} times")
    return _summarize(summary, model, tokenizer)

In [8]:
import glob
files = glob.glob("./data/texts/*.txt")
texts = defaultdict()
for afile in files:
      texts[re.sub('_','', afile.split('/')[3].split('.')[0])] = open(afile, encoding = 'utf-8').read()

In [9]:
content = texts['MSH01']

In [9]:
result = summarize(content, cache_dir = "/data/summaries/")

/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_token.py:99: UserWarning: 
Error while fetching `HF_TOKEN` secret value from your vault: 'Requesting secret HF_TOKEN timed out. Secrets can only be fetched when running from the Colab UI.'.
You are not authenticated with the Hugging Face Hub in this notebook.
If the error persists, please let us know by opening an issue on GitHub (https://github.com/huggingface/huggingface_hub/issues/new).
  warnings.warn(


tokenizer_config.json:   0%|          | 0.00/2.10k [00:00<?, ?B/s]

tokenizer.model:   0%|          | 0.00/493k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/1.80M [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/414 [00:00<?, ?B/s]

...

model-00002-of-00002.safetensors:   0%|          | 0.00/4.54G [00:00<?, ?B/s]

Loading checkpoint shards:   0%|          | 0/2 [00:00<?, ?it/s]

generation_config.json:   0%|          | 0.00/116 [00:00<?, ?B/s]

Token indices sequence length is longer than the specified maximum sequence length for this model (12910 > 1000). Running this sequence through the model will result in indexing errors
Setting `pad_token_id` to `eos_token_id`:2 for open-end generation.
/usr/local/lib/python3.10/dist-packages/transformers/generation/utils.py:1885: UserWarning: You are calling .generate() with the `input_ids` being on a device type different than your model's device. `input_ids` is on cpu, whereas the model is on cuda. You may experience unexpected behaviors or slower generation. Please make sure that you have put `input_ids` to the correct device by calling for example input_ids = input_ids.to('cuda') before running `.generate()`.
  warnings.warn(


KeyboardInterrupt: 

In [ ]:
print(f"summary for test doc that is {content.count(' ')} words, Approx {round(content.count(' ') / 500)} pages")
print(f"reduced to approximately {result.count(' ')} words")
print(result)
with open("sample_out.txt","w") as f:
    f.write(result)